# Reproducing Dublin pubs example

[Levi Wolf's Guinness delivery example](https://gist.github.com/ljwolf/e5927ab8c859ed477f496329c1ce19fc#file-guinness-py)

In [1]:
import geopandas as gpd
import pandas, numpy

In [2]:
import pandas

In [3]:
import sys
sys.path.insert(0, '/home/dylan/projects/gsoc2025/spopt/') # not the published spopt
import spopt

In [4]:
print(spopt.__file__)

/home/dylan/projects/gsoc2025/spopt/spopt/__init__.py


In [5]:
from spopt.route import engine, heuristic, utils

In [6]:
spopt.route.utils?

Type:        module
String form: <module 'spopt.route.utils' from '/home/dylan/projects/gsoc2025/spopt/spopt/route/utils.py'>
File:        ~/projects/gsoc2025/spopt/spopt/route/utils.py
Docstring:   <no docstring>

In [7]:
dir(spopt.route.engine)

['_OSRM_DATABASE_FILE',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_build_route_table_http',
 '_build_route_table_pyosrm',
 '_build_specific_route_http',
 '_build_specific_route_pyosrm',
 '_create_route_request',
 'build_route_table',
 'build_specific_route',
 'geopandas',
 'has_bindings',
 'metrics',
 'numpy',
 'os',
 'requests',
 'shapely',
 'warnings']

Reproduce the guinness example

In [8]:
from spopt.route.heuristic import LastMile
from pyvrp import stop

In [9]:
trucks = pandas.DataFrame(
    [['big', 'lng',      2000,    280, .004,  .50, 5],
     ['big', 'electric', 2000,    480, .002,  .50, 5],
     ['med', 'lng',      800, 280*.66, .0001, .63, 10],
     ['med', 'electric', 800, 480*.66, .004,  .50, 10],
     ['smo', 'lng',      400, 280*0.4, .002,  .50, 20],
     ['smo', 'electric', 400, 480*0.4, .0001, .63, 20],
     ],
     columns = [
         'namesize', 'namefuel', 'capacity', 
         'fixed_cost', 'cost_per_meter', 'cost_per_minute', 'n_truck'
         ]
)

In [10]:
dublin_pubs = gpd.read_file('/home/dylan/projects/gsoc2025/spopt/notebooks/gsoc2025/data/dublinpubs.geojson')

In [11]:
dublin_pubs.shape

(551, 8)

In [12]:
gdf = dublin_pubs

In [13]:
gdf.head()

,osmid,name,longitude,latitude,factype,demand,supply,geometry
0,0,Guinness Storehouse,-6.286880,53.341972,depot,NaN,NaN,POINT (-6.28688 53.34197)
1,133193958,The Barge Bar,-6.260588,53.330551,client,23.0,21.0,POINT (-6.26059 53.33055)
2,249426205,Marino House (Kavanagh's),-6.228816,53.365756,client,16.0,12.0,POINT (-6.22882 53.36576)
3,279464425,The Lower Deck,-6.265238,53.330686,client,28.0,5.0,POINT (-6.26524 53.33069)
4,279915988,Arlington Bar & Restaurant,-6.260933,53.347252,client,16.0,11.0,POINT (-6.26093 53.34725)


In [14]:
depot = gdf.iloc[0,:]
clients = gdf.iloc[1:,:].reset_index(drop=True)
clients = clients.set_index(clients.osmid.astype(str))

In [15]:
print('initializing model')
m = LastMile(
    depot_location=(depot.longitude.item(), depot.latitude.item()),
    depot_open=pandas.Timestamp("2030-01-02 07:00:00"),
    depot_close=pandas.Timestamp("2030-01-02 20:00:00"),
    depot_name=depot['name'],
)
print("adding clients")

m.add_clients(
    locations = clients.geometry, 
    delivery = clients.demand,
    pickup = clients.supply,
    time_windows=None,
    service_times=(numpy.log(clients.demand)**2).astype(int)
)
print("adding trucks")
m.add_trucks_from_frame(
    trucks, 
)
m.solve(stop=stop.MaxRuntime(60))
m.write_result("./guinness")

initializing model
adding clients
adding trucks


/home/dylan/projects/gsoc2025/spopt/spopt/route/engine.py:31: UserWarning: Failed to connect to routing engine... using haversine distance and (d/500)**.75 for durations
  warnings.warn(


PyVRP v0.11.1

Solving an instance with:
    1 depot
    550 clients
    70 vehicles (6 vehicle types)

                  |       Feasible        |      Infeasible
    Iters    Time |   #      Avg     Best |   #      Avg     Best

Search terminated in 60.29s after 166 iterations.
Best-found solution has cost 37985911.

Solution results
    # routes: 13
     # trips: 13
   # clients: 550
   objective: 37985911
    distance: 205951
    duration: 1117
# iterations: 166
    run-time: 60.29 seconds



NameError: name 'routing' is not defined